# Machine learning approch

In [2]:
from nltk.corpus import stopwords
import re
import pandas as pd
df = pd.read_csv('/home/hrishikeshkanojiya/Desktop/Cuad_labels.csv')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [3]:
df.head()

,Label Name,title,Context
0,Distributor Agreement,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,EXHIBIT 10.6\n\n ...
1,Promotion Agreement,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",Exhibit 10.26 CONFIDENTIAL TREATMENT HAS BE...
2,Supply Agreement,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,Exhibit 10.16 SUPPLY CONTRACT Contract No: Dat...
3,Hosting Agreement,CENTRACKINTERNATIONALINC_10_29_1999-EX-10.3-WE...,1 ...
4,Joint Venture Agreemen,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT,Exhibit 1\n\nJOINT FILING AGREEMENT\n\nThe und...


In [4]:
len(df['Label Name'].value_counts())

26

In [5]:
df['Label Name'].unique()

array(['Distributor Agreement', 'Promotion Agreement', 'Supply Agreement',
       'Hosting Agreement', 'Joint Venture Agreemen',
       'Endorsement Agreement', 'Consulting Agreement',
       'Marketing Agreement', 'Strategic Alliance Agreement',
       'Sponsorship Agreement', 'Franchise Agreement', 'IP Agreement',
       'Service Agreement', 'Agency Agreement', 'Maintenance Agreement',
       'License Agreement', 'Cooperation Agreement',
       'Collaboration/Cooperation Agreement', 'Reseller Agreement',
       'Outsourcing Agreement', 'Co-Branding Agreement',
       'Affiliate Agreement', 'Development Agreement',
       'Manufacturing Agreement', 'Transportation Agreement',
       'Non-Compete/No-Solicit/Non-Disparagement Agreement'], dtype=object)

In [6]:
df.isnull().sum()

Label Name    0
title         0
Context       0
dtype: int64

In [7]:
df.shape

(510, 3)

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.shape

(510, 3)

In [10]:
df['Context'].apply(lambda x: len(x.split(' '))).sum()

3361861

In [11]:
special_character_remover = re.compile('[/(){}\[\]\|@,;]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [32]:
def clean_text(text):
    text = text.lower()
    text = special_character_remover.sub(' ', text)
    text = extra_symbol_remover.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
    
df['Context'] = df['Context'].apply(clean_text)
df['Context'][8]


'exhibit 102______________________________________________________________________________copromotion agreementby betweendova pharmaceuticals incandvaleant pharmaceuticals north america llcseptember 26 2018______________________________________________________________________________confidential treatment requested portions exhibit copy filed herewith omits information subject confidentiality request omissions designated complete version exhibit filed separately securities exchange commissionsource dova pharmaceuticals inc 10q 11 8 2018table contentspagearticle 1 definitions 1article 2 rights obligations 821 engagement grant rights 822 retention rights 923 noncompetition nonsolicitation 924 dova trademarks copyrights 10article 3 joint steering committee 1131 formation jsc 1132 meetings minutes 1133 purpose jsc 1134 decision making 1335 marketing subcommittee 13article 4 valeant activities product 1441 valeant activities 1442 detailing 1543 compliance applicable law 1744 field force per

In [17]:
print(df['Context'].apply(lambda x: len(x.split(' '))).sum())

1605111


In [18]:
from sklearn.model_selection import train_test_split
X = df['Context']
y = df['Label Name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


In [19]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((408,), (102,), (408,), (102,))

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr.fit(X_train,y_train)
y_pred1 = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred1,y_test)}")

Accuracy is : 0.39215686274509803


# Bert Model

In [21]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer

/home/hrishikeshkanojiya/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from transformers import BertForSequenceClassification

In [23]:
from torch.utils.data import TensorDataset

In [24]:
possible_labels = df['Label Name'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Distributor Agreement': 0,
 'Promotion Agreement': 1,
 'Supply Agreement': 2,
 'Hosting Agreement': 3,
 'Joint Venture Agreemen': 4,
 'Endorsement Agreement': 5,
 'Consulting Agreement': 6,
 'Marketing Agreement': 7,
 'Strategic Alliance Agreement': 8,
 'Sponsorship Agreement': 9,
 'Franchise Agreement': 10,
 'IP Agreement': 11,
 'Service Agreement': 12,
 'Agency Agreement': 13,
 'Maintenance Agreement': 14,
 'License Agreement': 15,
 'Cooperation Agreement': 16,
 'Collaboration/Cooperation Agreement': 17,
 'Reseller Agreement': 18,
 'Outsourcing Agreement': 19,
 'Co-Branding Agreement': 20,
 'Affiliate Agreement': 21,
 'Development Agreement': 22,
 'Manufacturing Agreement': 23,
 'Transportation Agreement': 24,
 'Non-Compete/No-Solicit/Non-Disparagement Agreement': 25}

In [25]:
df['label'] = df['Label Name'].replace(label_dict)

In [27]:
df.head(20)

,Label Name,title,Context,label
0,Distributor Agreement,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,exhibit 106 distributor agreement distributor ...,0
1,Promotion Agreement,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",exhibit 1026 confidential treatment requested ...,1
2,Supply Agreement,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,exhibit 1016 supply contract contract date buy...,2
3,Hosting Agreement,CENTRACKINTERNATIONALINC_10_29_1999-EX-10.3-WE...,1 exhibit 103ion logo wwwioncom 5613949484 561...,3
4,Joint Venture Agreemen,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT,exhibit 1joint filing agreementthe undersigned...,4
5,Endorsement Agreement,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,redacted copyconfidential treatment requestedc...,5
6,Consulting Agreement,"KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...",exhibit 1023 corporate address fannin south pr...,6
7,Joint Venture Agreemen,"VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...",exhibit 1011amendment terminationofjoint ventu...,4
8,Promotion Agreement,DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_1...,exhibit 102___________________________________...,1
9,Marketing Agreement,"PACIRA PHARMACEUTICALS, INC. - A_R STRATEGIC L...",exhibit 1013confidential materials omitted fil...,7


In [29]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Label Name', 'label', 'data_type']).count()


title  \
Label Name                                         label data_type          
Affiliate Agreement                                21    train          9   
                                                         val            1   
Agency Agreement                                   13    train         11   
                                                         val            2   
Co-Branding Agreement                              20    train         19   
                                                         val            3   
Collaboration/Cooperation Agreement                17    train          9   
                                                         val            2   
Consulting Agreement                               6     train          9   
                                                         val            2   
Cooperation Agreement                              16    train         13   
                                                         val            2   
Development Agreement                              22    train         25   
                                                         val            4   
Distributor Agreement                              0     train         27   
                                                         val            5   
Endorsement Agreement                              5     train         20   
                                                         val            4   
Franchise Agreement                                10    train         13   
                                                         val            2   
Hosting Agreement                                  3     train         17   
                                                         val            3   
IP Agreement                                       11    train         15   
                                                         val            2   
Joint Venture Agreemen                             4     train         20   
                                                         val            3   
License Agreement                                  15    train         28   
                                                         val            5   
Maintenance Agreement                              14    train         29   
                                                         val            5   
Manufacturing Agreement                            23    train         14   
                                                         val            3   
Marketing Agreement                                7     train         14   
                                                         val            3   
Non-Compete/No-Solicit/Non-Disparagement Agreement 25    train          3   
Outsourcing Agreement                              19    train         15   
                                                         val            3   
Promotion Agreement                                1     train         10   
                                                         val            2   
Reseller Agreement                                 18    train         10   
                                                         val            2   
Service Agreement                                  12    train         24   
                                                         val            4   
Sponsorship Agreement                              9     train         26   
                                                         val            5   
Strategic Alliance Agreement                       8     train         27   
                                                         val            5   
Supply Agreement                                   2     train         15   
                                                         val            3   
Transportation Agreement                           24    train         11   
                                                         val    

In [30]:
df.head(20)

,Label Name,title,Context,label,data_type
0,Distributor Agreement,LIMEENERGYCO_09_09_1999-EX-10-DISTRIBUTOR AGRE...,exhibit 106 distributor agreement distributor ...,0,train
1,Promotion Agreement,"WHITESMOKE,INC_11_08_2011-EX-10.26-PROMOTION A...",exhibit 1026 confidential treatment requested ...,1,train
2,Supply Agreement,LohaCompanyltd_20191209_F-1_EX-10.16_11917878_...,exhibit 1016 supply contract contract date buy...,2,train
3,Hosting Agreement,CENTRACKINTERNATIONALINC_10_29_1999-EX-10.3-WE...,1 exhibit 103ion logo wwwioncom 5613949484 561...,3,train
4,Joint Venture Agreemen,NELNETINC_04_08_2020-EX-1-JOINT FILING AGREEMENT,exhibit 1joint filing agreementthe undersigned...,4,train
5,Endorsement Agreement,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,redacted copyconfidential treatment requestedc...,5,val
6,Consulting Agreement,"KIROMICBIOPHARMA,INC_05_11_2020-EX-10.23-CONSU...",exhibit 1023 corporate address fannin south pr...,6,val
7,Joint Venture Agreemen,"VEONEER,INC_02_21_2020-EX-10.11-JOINT VENTURE ...",exhibit 1011amendment terminationofjoint ventu...,4,train
8,Promotion Agreement,DovaPharmaceuticalsInc_20181108_10-Q_EX-10.2_1...,exhibit 102___________________________________...,1,train
9,Marketing Agreement,"PACIRA PHARMACEUTICALS, INC. - A_R STRATEGIC L...",exhibit 1013confidential materials omitted fil...,7,train


In [70]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

Downloading: 100%|████████████████████████████| 226k/226k [00:01<00:00, 204kB/s]
Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 22.0kB/s]
Downloading: 100%|██████████████████████████████| 570/570 [00:00<00:00, 421kB/s]


In [71]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Context.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/hrishikeshkanojiya/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [72]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Context.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

In [82]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)


In [83]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [84]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [85]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [86]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# nlp